In [1]:
import glob
import numpy as np
from batter import RBFESystem
from batter.input_process import get_configure_from_file, parse_input_file

from logging import getLogger, DEBUG 
logger = getLogger()
logger.setLevel(DEBUG)

/scratch/users/yuzhuang/miniforge3/envs/batter/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
protein_file = 'data/7T2G_mp/protein_input.pdb'
system_file = 'data/7T2G_mp/system_input.pdb'
ligand_files = ['data/7T2G_mp/ligand_MP.pdb',
                'data/7T2G_mp/ligand_MP2.pdb',
                'data/7T2G_mp/ligand_DAMGO.pdb']
#system_inpcrd = 'data/7T2G_mp/system_input.inpcrd'
equilibrated_rst = 'data/7T2G_mp/equilibrated.rst'

In [3]:
output_folder = 'test/rbfe'
test_system = RBFESystem(folder=output_folder)

10-Jan-2025 12:25:14 | INFO | Loading an existing system: /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/rbfe/
10-Jan-2025 12:25:15 | INFO | The folder does not contain fe: /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/rbfe/


In [4]:
test_system.create_system(
            system_name='7T2G',
            protein_input=protein_file,
            system_topology=system_file,
            system_coordinate=equilibrated_rst,
            ligand_paths=ligand_files,
#            overwrite=True,
            lipid_mol=['POPC'])

10-Jan-2025 12:25:17 | INFO | Input: membrane system
10-Jan-2025 12:25:17 | INFO | Getting the alignment of the protein and ligand to the system
10-Jan-2025 12:25:17 | WARNING | Elongating the ligand name: mp to mp1
10-Jan-2025 12:25:17 | WARNING | Elongating the ligand name: mp to mp2
10-Jan-2025 12:25:17 | WARNING | Shortening the ligand name: damg to dam
10-Jan-2025 12:25:17 | INFO | Preparing ligand poses
10-Jan-2025 12:25:18 | INFO | System loaded and prepared


In [5]:
import MDAnalysis as mda

u_prot = mda.Universe(f'{output_folder}/all-poses/reference.pdb')
u_lig = mda.Universe(f'{output_folder}/all-poses/pose0.pdb')
u_merge = mda.Merge(u_prot.atoms, u_lig.atoms)

P1_atom = u_merge.select_atoms('name CA and resid 149')
P2_atom = u_merge.select_atoms('name CA and resid 119')
P3_atom = u_merge.select_atoms('name CA and resid 328')
if P1_atom.n_atoms != 1 or P2_atom.n_atoms != 1 or P3_atom.n_atoms != 1:
    raise ValueError('Error: more than one atom selected')

lig_atom = u_merge.select_atoms(f'resname {u_lig.atoms.resnames[0]} and name C12')

# get ll_x,y,z distances
r_vect = lig_atom.center_of_mass() - P1_atom.positions
print(f'l1_x: {r_vect[0][0]:.2f}')
print(f'l1_y: {r_vect[0][1]:.2f}')
print(f'l1_z: {r_vect[0][2]:.2f}')

l1_x: 1.98
l1_y: -6.75
l1_z: 4.06


Modify the `test/abfe/abfe.in` file to include the following lines based on the output of the above script:
    
```bash
    l1_x: xxx
    l1_y: xxx
    l1_z: xxx
```

In [6]:
sim_config = get_configure_from_file('data/md_input/lipid_rbfe.in')

In [7]:
# Prepare the system for the equilibration stage
test_system.prepare(
    stage='equil',
#    input_file='data/md_input/lipid.in',
    input_file=sim_config,
    overwrite=False,
)

10-Jan-2025 12:25:31 | INFO | Preparing the system
10-Jan-2025 12:25:31 | INFO | Simulation configuration: software='amber' calc_type='dock' celpp_receptor='7T2G' poses_list=['0', '1', '2'] p1=':147@CA' p2=':117@CA' p3=':326@CA' ligand_list=[] other_mol=[] solv_shell=4.0 lipid_mol=['POPC', 'PA', 'PC', 'OL'] fe_type='relative' components=['x', 'e', 'n', 'm'] release_eq=[10.0, 2.5, 0.5, 0.0] attach_rest=[0.0, 0.1, 0.24, 0.56, 1.33, 3.16, 7.5, 10.5, 17.78, 20.0, 42.17, 50.0, 60.0, 75.0, 80.0, 100.0] ti_points=0 lambdas=[0.0001, 0.02, 0.04, 0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.92, 0.96, 0.98, 0.9999] sdr_dist=40.0 dec_method='exchange' dec_int='mbar' blocks=5 rec_dihcf_force=50.0 rec_discf_force=5.0 lig_distance_force=5.0 lig_angle_force=250.0 lig_dihcf_force=70.0 rec_com_force=10.0 lig_com_force=10.0 water_model='TIP3P' num_waters=0 buffer_x=12.0 buffer_y=12.0 buffer_z=22.0 lig_buffer=15.0 neutralize_only='no' cation='Na+' anion='

Building the system in /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/rbfe/equil/pose0


10-Jan-2025 12:25:45 | INFO | Ligand anchors:
10-Jan-2025 12:25:45 | INFO | L1: :289@C24
10-Jan-2025 12:25:45 | INFO | L2: :289@C49
10-Jan-2025 12:25:45 | INFO | L3: :289@O51
10-Jan-2025 12:25:45 | WARNING | WARNING: Switch to Berendsen barostat
10-Jan-2025 12:26:17 | INFO | Preparing pose: pose1
10-Jan-2025 12:26:17 | INFO | Building pose1...
10-Jan-2025 12:26:19 | WARNING | Receptor is not set as chain A; trying to fetch the first residue from the protein and it may be wrong.
10-Jan-2025 12:26:19 | INFO | P1: :82@CA
10-Jan-2025 12:26:19 | INFO | P2: :52@CA
10-Jan-2025 12:26:19 | INFO | P3: :261@CA
10-Jan-2025 12:26:19 | WARNING | The residue names of the lipids are four-letter names.They were truncated to three-letter namesfor compatibility with AMBER.
10-Jan-2025 12:26:29 | INFO | Ligand anchors:
10-Jan-2025 12:26:29 | INFO | L1: :289@C24
10-Jan-2025 12:26:29 | INFO | L2: :289@C49
10-Jan-2025 12:26:29 | INFO | L3: :289@O51


Building the system in /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/rbfe/equil/pose1


10-Jan-2025 12:26:29 | WARNING | WARNING: Switch to Berendsen barostat
10-Jan-2025 12:27:01 | INFO | Preparing pose: pose2
10-Jan-2025 12:27:01 | INFO | Building pose2...
10-Jan-2025 12:27:03 | WARNING | Receptor is not set as chain A; trying to fetch the first residue from the protein and it may be wrong.
10-Jan-2025 12:27:03 | INFO | P1: :82@CA
10-Jan-2025 12:27:03 | INFO | P2: :52@CA
10-Jan-2025 12:27:03 | INFO | P3: :261@CA
10-Jan-2025 12:27:03 | WARNING | The residue names of the lipids are four-letter names.They were truncated to three-letter namesfor compatibility with AMBER.


Building the system in /oak/stanford/groups/rondror/users/yuzhuang/software/batter/examples/test/rbfe/equil/pose2


10-Jan-2025 12:27:14 | INFO | Ligand anchors:
10-Jan-2025 12:27:14 | INFO | L1: :289@C16
10-Jan-2025 12:27:14 | INFO | L2: :289@N
10-Jan-2025 12:27:14 | INFO | L3: :289@C10
10-Jan-2025 12:27:14 | WARNING | WARNING: Switch to Berendsen barostat
10-Jan-2025 12:27:45 | INFO | Equilibration systems have been created for all poses listed in the input file.
10-Jan-2025 12:27:45 | INFO | now cd equil/pose0
10-Jan-2025 12:27:45 | INFO | sbatch SLURMM-run
10-Jan-2025 12:27:45 | INFO | Equil System prepared


In [9]:
# Submit the equilibration job to SLURM
test_system.submit(stage='equil')

09-Jan-2025 15:08:41 | INFO | Submit equilibration stage
09-Jan-2025 15:08:43 | INFO | Equilibration systems have been submitted for all poses listed in the input file.


In [8]:
# Prepare the system for the free energy calculation
test_system.prepare(
    stage='fe',
#    input_file='data/md_input/lipid_rbfe.in',
    input_file=sim_config,
#    overwrite=True,
)

10-Jan-2025 12:34:26 | INFO | Preparing the system
10-Jan-2025 12:34:26 | INFO | Simulation configuration: software='amber' calc_type='dock' celpp_receptor='7T2G' poses_list=['0', '1', '2'] p1=':147@CA' p2=':117@CA' p3=':326@CA' ligand_list=[] other_mol=[] solv_shell=4.0 lipid_mol=['POPC', 'PA', 'PC', 'OL'] fe_type='relative' components=['x', 'e', 'n', 'm'] release_eq=[10.0, 2.5, 0.5, 0.0] attach_rest=[0.0, 0.1, 0.24, 0.56, 1.33, 3.16, 7.5, 10.5, 17.78, 20.0, 42.17, 50.0, 60.0, 75.0, 80.0, 100.0] ti_points=0 lambdas=[0.0001, 0.02, 0.04, 0.06, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.92, 0.96, 0.98, 0.9999] sdr_dist=40.0 dec_method='exchange' dec_int='mbar' blocks=5 rec_dihcf_force=50.0 rec_discf_force=5.0 lig_distance_force=5.0 lig_angle_force=250.0 lig_dihcf_force=70.0 rec_com_force=10.0 lig_com_force=10.0 water_model='TIP3P' num_waters=0 buffer_x=12.0 buffer_y=12.0 buffer_z=22.0 lig_buffer=15.0 neutralize_only='no' cation='Na+' anion='

NotImplementedError: Free energy system preparation is not implemented yet

In [22]:
import MDAnalysis as mda
u = mda.Universe('./data/7T2G_mp/ligand_DAMGO.pdb')

In [23]:
from MDAnalysis.guesser.default_guesser import DefaultGuesser

In [24]:
u.guess_TopologyAttrs(to_guess=['elements'])

In [25]:
u.add_TopologyAttr('charges')

In [26]:
u.atoms.charges = DefaultGuesser(u).guess_gasteiger_charges(u.atoms)

In [5]:
u.atoms.charges.sum()

NameError: name 'u' is not defined